## ✨ There's a little magic in everything ✨

In [ ]:
obj = object()

That called `object.__init__`

In [ ]:
obj

That called `obj.__repr__`

In [ ]:
str(obj)

That called `obj.__str__` (which calls `obj.__repr__` as a fallback)

```
















```

## Stringy Types

Implementing [`__repr__`](https://docs.python.org/3/reference/datamodel.html#object.__repr__) and [`__str__`](https://docs.python.org/3/reference/datamodel.html#object.__str__) can often be useful when debugging or manipulating objects interactively (like via Notebooks or REPLs)

In [ ]:
class Stringy:
    def __init__(self, name):
        self.name = name

    def __repr__(self) -> str:
        """
        ...If at all possible, this should look like a valid Python expression 
        that could be used to recreate an object with the same value. If this 
        is not possible, a string of the form <...some useful description...> 
        should be returned. 
        """
        return f"{self.__class__.__name__}('{self.name}')"
    
    def __str__(self) -> str:
        """        
        This method differs from `__repr__()` in that there is no expectation 
        that `__str__()` returns a valid Python expression: a more convenient 
        or concise representation can be used.
        """
        return f"Name: {self.name}"

In [ ]:
a = Stringy('apple')
a

In [ ]:
repr(a)

In [ ]:
str(a)

---

Why not something like:

In [ ]:
class StringyToo:
    def __init__(self, name):
        self.name = name
        
    def to_string(self):
        return f"Name: {self.name}"

In [ ]:
b = StringyToo('cheese')
b

In [ ]:
b.to_string()

```
















```

### Quacks like...

String-like objects can be used in place of other strings

In [ ]:
f"I have an object. {b}."

In [ ]:
f"I have an object. {a}. It was made with: {a!r}"

```
















```

In [ ]:
def verbose(method):
    """Wrapper to print the method name, arguments, and result of a call"""
    def _method(self, *args):
        arg_str = ', '.join(repr(a) for a in args)
        result = method(self, *args)
        print(f'call: {self!r}.{method.__name__}({arg_str}) => {result!r}')
        return result
    return _method

## Math-able Types

Implementing the mathematical magic methods can let you use any of the infix operators to manipulate your objects as makes sense.

    x + y     __add__             x & y   __and__
    x - y     __sub__             x | y   __or__      
    x * y     __mul__             x ^ y   __xor__
    x / y     __truediv__         x << y  __lshift__
    x // y    __floordiv__        x >> y  __rshift__
    x ** y    __pow__
    x % y     __mod__
    x @ y     __matmul__

In [ ]:
class PathyBase:
    def __init__(self, path):
        self.path = path

    def __repr__(self):
        return f"{self.__class__.__name__}({self.path!r})"


class PathA(PathyBase):
    @verbose
    def __truediv__(self, other) -> str:
        return self.path + '/' + other.path

In [ ]:
PathA('aaa') / PathA('bbb')

In [ ]:
PathA('aaa').__truediv__(PathA('bbb'))

---

In [ ]:
class PathB(PathyBase):
    @verbose
    def __truediv__(self, other) -> int:
        if isinstance(other, self.__class__):
            other_value = other.path
        else:
            other_value = other

        return self.__class__(self.path + '/' + other_value)

In [ ]:
PathB('ccc') / 'ddd'

In [ ]:
# doesn't work
'ccc' / PathB('ddd')

In [ ]:
# also doesn't work (same error)
result = 'usr' / PathB('bin') / 'python'

In [ ]:
# equivalent
_intermediate: str = 'usr' / PathB('bin')
result = _intermediate / 'python'

---

To support those operand type(s), we use the [*reflected* methods](https://docs.python.org/3/reference/datamodel.html?#object.__radd__), `__r*__`. When the "forward" direction fails, Python will attempt to call it the other way, so `'usr' + PathB('lib')` is effectively something like:

```python
try:
    'usr'.__truediv__(PathB('lib'))   # the str type doesn't know what a Path is
except TypeError:
    PathB('lib').__rtruediv__('usr')  # but we have the option to do something!
```

In [ ]:
class PathC(PathB):
    # __truediv__ from PathB

    @verbose
    def __rtruediv__(self, other) -> str:
        """
        Note that the order of arguments are reversed on the call; 
        so `self` will be the object *after* the infix operator, and 
        `other` the one before.
        
        We're also assuming that other is an integer, Path / Path
        operations will just use __truediv__ from the first one.
        """
        return self.__class__(other + '/' + self.path)

In [ ]:
'bin' / PathC('bash')

In [ ]:
PathC('home') / PathC('nick')

In [ ]:
PathC('home') / 'nick' / PathC('Code')

In [ ]:
PathC('home') / 'nick' / 'Code' / 'offhours/chipy'

### Abstract Mixins

The built-in `open()` accepts objects that are [`os.PathLike`](https://docs.python.org/3/library/os.html#os.PathLike). `PathLike` is an **abstract base class** that objects can define to specify how they are path-like.

In [ ]:
import os

class PathD(PathC, os.PathLike):
    @verbose
    def __fspath__(self):
        return os.path.expanduser(self.path)

In [ ]:
print('creating path')
ignore = PathD('~') / '.gitignore_global'

print('opening it!')
with open(ignore) as f:
    print('\n==============\nFile Contents:\n==============')
    for line in f:
        print(line)

We've created a hacky version of [`pathlib`](https://docs.python.org/3/library/pathlib.html), a standard library package.  (Use that one instead.)

```
















```

## Orderable Types

In [ ]:
versions = [
    '0.9.4',
    '0.95.3',
    '0.10.5',
    '0.9.34',
    '0.10.42',
    '1.4.2',
]

In [ ]:
sorted(versions)

How can we sort these? We could pass a `key` argument into `sorted()`:

In [ ]:
sorted(versions, key=lambda i: [int(p) for p in i.split('.')])

In [ ]:
class Version:
    def __init__(self, version):
        self.version = version
        self.major, self.minor, self.patch = (int(x) for x in version.split('.'))
    
    def __repr__(self):
        return f'{self.__class__.__name__}({self.version!r})'

    @verbose
    def __lt__(self, other):
        return (
            (self.major, self.minor, self.patch) 
            < (other.major, other.minor, other.patch)
        )

In [ ]:
Version('0.32.52') < Version('0.93.2')

In [ ]:
Version('10.0.0') < Version('9.99.999')

In [ ]:
version_objs = [Version(v) for v in versions]
version_objs

In [ ]:
sorted(version_objs)

In [ ]:
max(version_objs)

In [ ]:
min(version_objs)

Also, not everything can take a `key` argument:

In [ ]:
import heapq

version_heap = version_objs[:]
heapq.heapify(version_heap)

In [ ]:
while version_heap:
    print(heapq.heappop(version_heap))

```
















```

## Container Methods

In [ ]:
from contextlib import suppress

class FallbackDict:
    """Look through a series of dicts in order"""
    def __init__(self, *dicts):
        self.dicts = dicts

    def __getitem__(self, key):
        for d in self.dicts:
            with suppress(KeyError):  
                return d[key]
        raise KeyError(key)

In [ ]:
dicts = [
    {"name": "chipy", "misc": "woot"},
    {"misc": "etc"},
    {"name": "unkown", "greeting": "hello"},
]
fbd = FallbackDict(*dicts)

f'{fbd["greeting"]}, {fbd["name"]}!'

It's kinda like a dictionary, we can index into it and get items, but not much else.

In [ ]:
len(fbd)  # probably should be 3?

In [ ]:
[k for k in fbd.values()]  # should be 'woot', 'hello', 'chipy'

In [ ]:
'name' in fbd  # is a key in it?

In [ ]:
# would be nice to convert to a normal dict if we wanted to JSON serialize it
dict(fbd)

### [Abstract Base Classes for Containers](https://docs.python.org/3/library/collections.abc.html) (`collections.abc`)

Amplify your magics! If you implement the abstract methods required by the type, it can provide additional mixin methods for free. Lets make our `FallbackDict` more `dict`-like by using the `Mapping` ABC (read-only dict)

In [ ]:
import collections.abc


class FallbackDictM(FallbackDict, collections.abc.Mapping):
    # __init__, __getitem__ from FallbackDict
    
    def _all_keys(self):
        all_keys = set()
        for d in self.dicts:
            all_keys |= set(d)
        return all_keys
    
    def __len__(self):
        return len(self._all_keys())

    def __iter__(self):
        for key in self._all_keys():
            yield key

In [ ]:
fbdm = FallbackDictM(*dicts)

In [ ]:
print(len(fbdm))  # directly hitting __len__
for key in fbdm:  # hits __iter__
    print(key, fbdm[key])

In [ ]:
list(fbdm.values())  # values method for free

In [ ]:
'name' in fbdm  # __contains__ for free

In [ ]:
flattened = dict(fbdm)
flattened

In [ ]:
flattened == fbdm  # __eq__ for free

```
















```

In [ ]:
import re
import requests

def get_xkcd_hotlink(comic_number):
    """Extract the embedded URL for an XKCD comic (no magic here)"""
    match = re.search(
        r"Image URL \(for hotlinking/embedding\):\s*(?P<url>http.*(jpg|png))", 
        requests.get(f"https://xkcd.com/{comic_number}/").text,
    )
    if match:
        return match.groupdict()["url"]

## Proprietary Magic

https://ipython.readthedocs.io/en/stable/config/integrating.html#rich-display

IPython notebook can display richer representations of objects. To use this, you can define any of a number of `_repr_*_()` methods. Note that these are surrounded by single, not double underscores.

The notebook can display `svg`, `png`, `jpeg`, `html`, `javascript`, `markdown` and `latex`. If the methods don’t exist, or return `None`, it falls back to a standard `repr()`.

In [ ]:
class XKCDViewer:        
    def __getitem__(self, key):
        url = get_xkcd_hotlink(key)
        if url is None:
            raise KeyError("couldn't find XKCD comic with that number")
        return XKCDComic(url)


class XKCDComic:
    def __init__(self, url):
        self.url = url

    def _repr_html_(self):
        return f"<img src='{self.url}'>"


xkcd = XKCDViewer()

In [ ]:
ANTIGRAVITY = 353
BOBBY_TABLES = 327
PYTHON_ENV = 1987
SANDWICH = 149

xkcd[ANTIGRAVITY]

```
















```

## Bonus: Context Managers

In [ ]:
class Suppressor:
    """
    https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers
    """
    def __init__(self, exc_ignore):
        self.exc_ignore = exc_ignore

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        """
        ... If an exception is supplied, and the method wishes to suppress the 
        exception, it should return a true value. ...
        """
        return (
            exc_type is not None 
            and issubclass(exc_type, self.exc_ignore)
        )

In [ ]:
with Suppressor(IndexError):
    print('abcde'[:3])  # works normally

with Suppressor(IndexError):
    print('abcde'[1000])  # raises an error, but it gets suppressed
    
print('mmmkay')

In [ ]:
with Suppressor(RuntimeError):
    'abcde'[1000]  # raises the wrong error

We basically re-implemented [`contextlib.suppress`](https://docs.python.org/3/library/contextlib.html#contextlib.suppress) [(source)](https://github.com/python/cpython/blob/cca4eec3c0/Lib/contextlib.py#L342) from the standard lib.

In [ ]:
import traceback

class expect_exception:
    """
    Expect an exception and trap it. Helps run all of a notebook
    that may have intentional errors for pedagogical reasons
    """
    def __init__(self, *exc_types):
        self.exc_types = exc_types
        
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, tb):
        if exc_type is None:
            raise RuntimeError('expected exception did not occur!')
        if issubclass(exc_type, self.exc_types):
            traceback.print_exception(exc_type, exc_value, tb)
            return True

In [ ]:
with expect_exception(Exception):
    PathA('a') / 'b'